<table style='border: none' align='left'>
   <tr style='border: none'>
      <th style='border: none'><font face='verdana' size='5' color='black'><b>Export and import a deployment space using the WML Python client</b></th>
      <th style='border: none'><img src='https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true' alt='Watson Machine Learning icon' height='40' width='40'></th>
   </tr>
</table>

This notebook demonstrates how to use the Watson Machine Learning Python client (V4) to export and import a deployment space. It's compatible with CP4D 3.0 and Python 3.6.

## Table of Contents

This notebook contains the following parts:
1. [Set up the environment](#setup)
2. [Populate the deployment space](#populate)
3. [Export the deployment space](#export)
4. [Import the deployment space](#import)
5. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

To get started on CP4D 3.0, find documentation on installation and set up <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/cpd/overview/welcome.html" target="_blank" rel="noopener no referrer">here</a>.

Import the `watson-machine-learning-client` module.
<div class="alert alert-block alert-info">
For more information about the <b>Watson Machine Learning Python client (V4)</b>, please refer to the <a href="https://wml-api-pyclient-dev-v4.mybluemix.net/" target="_blank" rel="noopener no referrer">Python client documentation</a>. If you're using the notebook within a project on your CP4D cluster, you do not need to install this package as it comes pre-installed with the notebooks. The installation code below is for demonstration but is non-executable at this stage.
</div>

In [1]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**Authenticate the Python client on CP4D.**

<div class="alert alert-block alert-info">To find your authentication information (your credentials) follow the steps provided here in the <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/ml-authentication-local.html" target="_blank" rel="noopener no referrer">Documentation.</a></div>

In [2]:
# Enter your credentials here.

from project_lib.utils import environment
url = environment.get_common_api_url()

import sys,os,os.path
token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
     "instance_id": "openshift",
     "token": token,
     "url": url,
     "version": "3.0.0"
}

In [3]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [4]:
client.version

'1.0.64'

### To set the default space, follow these steps.

<div class="alert alert-block alert-info">
You can create your own <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/ml-spaces_local.html" target="_blank" rel="noopener no referrer">deployment space</a> by selecting <b>Analytics deployments</b> under <b>Analyze</b> from the Navigation Menu on the top left of this page. Alternatively, you can create a deployment and obtain its UID using the code in the following cell.</div>

Enter the name of the space in the metadata.

In [5]:
# Enter the name of your deployment space here:
space_details = client.spaces.store(meta_props={client.spaces.ConfigurationMetaNames.NAME: 'YOUR DEPLOYMENT SPACE'})
space_uid = client.spaces.get_uid(space_details)
print('Space UID: {}'.format(space_uid))

Space UID: 8a4a090c-96cc-4fac-932b-02bfad1455fc


You can set the default space using the cell below.

In [6]:
client.set.default_space(space_uid)

'SUCCESS'

<a id="populate"></a>
## 2. Populate the deployment space

Download the following file to store in the deployment space.

In [7]:
!wget https://github.com/IBMDataScience/sample-notebooks/raw/master/Files/transform_pred_script.py --output-document=transform_pred_script.py

--2020-05-12 06:07:20--  https://github.com/IBMDataScience/sample-notebooks/raw/master/Files/transform_pred_script.py
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBMDataScience/sample-notebooks/master/Files/transform_pred_script.py [following]
--2020-05-12 06:07:21--  https://raw.githubusercontent.com/IBMDataScience/sample-notebooks/master/Files/transform_pred_script.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.36.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.36.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1716 (1.7K) [text/plain]
Saving to: ‘transform_pred_script.py’

100%[======================================>] 1,716       --.-K/s   in 0s      

2020-05-12 06:07:21 (12.6 MB/s) - ‘transform_pred_script.py’ saved [1716/

Create the script metadata and store the script in the deployment space.

In [8]:
# Script metadata.
sw_spec_uid = client.software_specifications.get_uid_by_name('ai-function_0.1-py3.6')

script_meta_props = {
    client.script.ConfigurationMetaNames.NAME: 'My_Script',
    client.script.ConfigurationMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid
}

In [9]:
# Create the asset for the script.
from pprint import pprint
script_details = client.script.store(script_meta_props, 'transform_pred_script.py')
pprint(script_details)

Creating Script asset...
SUCCESS
{'entity': {'script': {'language': {'name': 'python3'},
                       'software_spec': {'base_id': '0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda'}}},
 'metadata': {'asset_type': 'script',
              'created_at': '2020-05-12T06:07:24Z',
              'guid': '1c2cdd9d-936b-47a6-84ef-0bb53283593d',
              'href': '/v2/assets/1c2cdd9d-936b-47a6-84ef-0bb53283593d?space_id=8a4a090c-96cc-4fac-932b-02bfad1455fc',
              'last_updated_at': '2020-05-12T06:07:24Z',
              'space_id': '8a4a090c-96cc-4fac-932b-02bfad1455fc'}}


List the scripts using the following code.

In [10]:
client.script.list()

---------  ----------  ------------------------------------
NAME       ASSET_TYPE  ASSET_ID
My_Script  script      1c2cdd9d-936b-47a6-84ef-0bb53283593d
---------  ----------  ------------------------------------


<div class="alert alert-block alert-info">
From the list above, you can see that the script was successfully saved in the deployment space.</div>

Get the UID of the script.

In [11]:
# Script UID.
script_uid = client.script.get_uid(script_details)
print('Script UID: {}'.format(script_uid))

Script UID: 1c2cdd9d-936b-47a6-84ef-0bb53283593d


<a id="export"></a>
## 3. Export the deployment space

Create the metadata to export the deployment space with the script from the previous sections.

In [12]:
# Create the export.
import time
exports_meta_props = {
    client.spaces.ExportMetaNames.NAME: 'Export request started at ' + time.asctime(),
    client.spaces.ExportMetaNames.DESCRIPTION: 'Description of exports',  # optional
    # use "wml_model" for models
    client.spaces.ExportMetaNames.ASSETS: {'wml_function': [script_uid]}
}

Create the export and get the UID.

In [13]:
export_details = client.spaces.exports(space_uid, meta_props=exports_meta_props)
exports_uid = client.spaces.get_exports_uid(export_details)
print('Export UID: {}'.format(exports_uid))

Export UID: 46dd9b40-f8ce-4f64-8539-e22343163568


You can use the following code to track the status of the export until it is complete.

In [14]:
export_status = export_details['entity']['status']
eloop = 0
while export_status['state'] != 'completed':
    if export_status['state'] == 'failed':
        break
    print('Export status: {}'.format(export_status))
    eloop += 1
    if eloop > 30:
        break
    time.sleep(5)
    export_status = client.spaces.get_exports_details(space_uid, exports_uid)['entity']['status']
print('Export details: {}'.format(client.spaces.get_exports_details(space_uid, exports_uid)))

Export status: {'state': 'pending'}
Export details: {'metadata': {'name': 'Export request started at Tue May 12 06:07:25 2020', 'guid': '46dd9b40-f8ce-4f64-8539-e22343163568', 'description': 'Description of exports', 'id': '46dd9b40-f8ce-4f64-8539-e22343163568', 'modified_at': '2020-05-12T06:07:29.508Z', 'created_at': '2020-05-12T06:07:26.776Z', 'href': '/v4/spaces/8a4a090c-96cc-4fac-932b-02bfad1455fc/exports/46dd9b40-f8ce-4f64-8539-e22343163568', 'space_id': '8a4a090c-96cc-4fac-932b-02bfad1455fc'}, 'entity': {'status': {'state': 'completed'}, 'assets': {'wml_function': ['1c2cdd9d-936b-47a6-84ef-0bb53283593d']}}}


Download the exported deployment space.

In [15]:
path = client.spaces.download(space_uid, exports_uid)
print('Downloaded export zip: {}'.format(path))



--------------------------------------------------------
Successfully downloaded spaces export file: wmlspace.zip
--------------------------------------------------------


Downloaded export zip: wmlspace.zip


<a id="import"></a>
## 4. Import the new deployment space

Create the metadata of the new deployment space. Enter the name of the space in the metadata.

In [16]:
# Create the import. Enter the name of your new deployment space here:
import_metadata = {
    client.spaces.ConfigurationMetaNames.NAME: 'YOUR IMPORT SPACE'
}

Once you store the new space, get the UID.

In [17]:
import_space_details = client.spaces.store(meta_props=import_metadata)
import_space_uid = client.spaces.get_uid(import_space_details)
print('Import space UID: {}'.format(import_space_uid))

Import space UID: de551dbc-40f6-416b-84f8-6e9173c02404


Import the exported .zip file from the previous section into the new deployment space. This will import the assets (in this case, the script) from the exported deployment space.

In [18]:
# Import the export zip.
import_details = client.spaces.imports(import_space_uid, file_path=path)
imports_uid = client.spaces.get_imports_uid(import_details)
print('Import UID: {}'.format(imports_uid))

Import UID: 704c8ce3-6177-4693-9d8a-44b28223be4e


You can use the following code to track the status of the import until it is complete.

In [19]:
# wait for the import to finish
import_status = import_details['entity']['status']
iloop = 0
while import_status['state'] != 'completed':
    if import_status['state'] == 'failed':
        break
    print('Import status: {}'.format(import_status))
    iloop += 1
    if iloop > 30:
        break
    time.sleep(5)
    import_status = client.spaces.get_imports_details(import_space_uid, imports_uid)['entity']['status']
print('Import details: {}'.format(client.spaces.get_imports_details(import_space_uid, imports_uid)))

Import status: {'state': 'pending'}
Import status: {'state': 'pending'}
Import details: {'metadata': {'guid': '704c8ce3-6177-4693-9d8a-44b28223be4e', 'id': '704c8ce3-6177-4693-9d8a-44b28223be4e', 'modified_at': '2020-05-12T06:07:40.716Z', 'created_at': '2020-05-12T06:07:35.079Z', 'href': '/v4/spaces/de551dbc-40f6-416b-84f8-6e9173c02404/imports/704c8ce3-6177-4693-9d8a-44b28223be4e', 'space_id': 'de551dbc-40f6-416b-84f8-6e9173c02404'}, 'entity': {'status': {'state': 'completed'}}}


Set this deployment space as the new default space.

In [20]:
# check that the function has been imported
client.set.default_space(import_space_uid)

'SUCCESS'

List the scripts in the new space.

In [21]:
# show the script assets
client.script.list()

---------  ----------  ------------------------------------
NAME       ASSET_TYPE  ASSET_ID
My_Script  script      890f30c0-ff2e-4e2a-a228-043a7aaa3154
---------  ----------  ------------------------------------


<div class="alert alert-block alert-info">
From the list above, you can see that the script was successfully imported into the deployment space.</div>

## 5. Summary and next steps <a id="summary"></a>

You have successfully completed this notebook!

You learned how to use the Python client and import and export deployment spaces.

### Resources <a id="resources"></a>

To learn more about configurations used in this notebook or more sample notebooks, tutorials, documentation, how-tos, and blog posts, check out these links:

<div class="alert alert-block alert-success">

<h4>IBM documentation</h4>
<br>
 <li> <a href="https://wml-api-pyclient-dev-v4.mybluemix.net" target="_blank" rel="noopener no referrer">watson-machine-learning</a></li> 
 <li> <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/cpd/overview/welcome.html" target="_blank" rel="noopener noreferrer">CP4D 3.0</a></li>
 <li> <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/ml-spaces_local.html" target="_blank" rel="noopener noreferrer">Deployment Spaces</a></li>
 
<h4> IBM Samples</h4>
<br>
 <li> <a href="https://github.com/IBMDataScience/sample-notebooks" target="_blank" rel="noopener noreferrer">Sample notebooks</a></li>
 
<h4> Others</h4>
 <li> <a href="https://www.python.org" target="_blank" rel="noopener noreferrer">Official Python website</a></li>
 </div>

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style='background:#F5F7FA; height:110px; padding: 2em; font-size:14px;'>
<span style='font-size:18px;color:#152935;'>Love this notebook? </span>
<span style='font-size:15px;color:#152935;float:right;margin-right:40px;'>Don't have an account yet?</span><br>
<span style='color:#5A6872;'>Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style='border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;'><a href='https://ibm.co/wsnotebooks' target='_blank' style='color: #3d70b2;text-decoration: none;'>Sign Up</a></span><br>
</div>